In [ ]:
import mlflow
import pandas as pd

uri = ... # Set your MLflow tracking URI here
mlflow.set_tracking_uri(uri)


In [ ]:


def generate_time_table(experiment_ids):
    all_rows = []

    for exp_id in experiment_ids:
        runs = mlflow.search_runs(
            experiment_ids=[exp_id],
            output_format="list"
        )
        for run in runs:
            if run.data.params.get("group_set") != "test" or run.data.params.get("group_type") != "sim":
                continue

            dataset = run.data.params.get("dataset", f"Exp-{exp_id}")
            approach = run.data.params.get("recommender_strategy", None)
            aggregation = run.data.params.get("SAE_fusion_strategy", 'none')
            
            
            row_key = (approach, aggregation)
            metrics = {
                dataset: run.data.metrics.get("Time/mean", 0),
            }

            all_rows.append((row_key, metrics))

    # Build DataFrame from records
    records = {}
    for key, metrics in all_rows:
        if key not in records:
            records[key] = {}
        records[key].update(metrics)

    df = pd.DataFrame.from_dict(records, orient="index")
    df.index.names = ["Approach", "Aggregation"]

    # Sort and reindex columns by dataset then metric
    df = df.sort_index(axis=1, level=[0, 1]).sort_values(
        by=["Approach", "Aggregation"]
    )
    
    dataset_order = ["MovieLens", "LastFM1k"]

    cols = df.columns
    cols = sorted(
        cols,
        key=lambda x: (
            dataset_order.index(x) if x in dataset_order else len(dataset_order),
        )
    )
    df = df[cols]

    return df.reset_index()

# Comparing with other approaches

## Group Recommendations Results for **Similar** groups sorted by MovieLens G/mean

In [ ]:
experiments = ['523100174176986081', '333391697323445885']

# Select only the desired columns for aggregation


table = generate_time_table(experiments)
table = table[~table['Aggregation'].isin(["topk", "common_features", "wcom", "max"])]
# get time value for SAE for both datasets
sae_times = table[table['Approach'] == "SAE"].set_index('Aggregation').loc[:, ["MovieLens", "LastFM1k"]].values.flatten()
print(sae_times)
# normalize the times
table["MovieLens"] = table["MovieLens"] / sae_times[0]
table["LastFM1k"] = table["LastFM1k"] / sae_times[1]
selected_columns = ["MovieLens", "LastFM1k"]
    


table = table.sort_values(
    by=["Approach","Aggregation"], ascending=True
).drop(columns=["Aggregation"])

[0.00369252 0.00332541]


In [7]:
table

,Approach,MovieLens,LastFM1k
0,ADD,16.848631,18.484187
1,ELSA,0.324075,0.812582
2,ELSA_INT,0.756003,0.650647
3,EPFuzzDA,32.119747,35.039343
4,GFAR,46.347114,51.535524
5,LMS,17.249598,18.750383
6,MPL,18.433887,20.246800
7,SAE,1.000000,1.000000


In [8]:
table.to_latex(
    "sae_table.tex",
    index=False,
    float_format="%.1f",
    bold_rows=False,
    column_format="l|rr",
    escape=False,
    caption = (
        "Table with inference times for group RS approaches. The time is normalized to the SAE approach for each dataset. "
    ),
    label="tab:time"
)